In [ ]:
# default_exp model.dummy

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import albumentations as A
import albumentations.pytorch.transforms as AT

from pytorch_lightning.metrics.functional import accuracy
from fractal_augmentation.data.cifar import CIFAR10DataModule, CIFAR10
from fractal_augmentation.transforms.fractal import MandelBrotFractalTransform
from loguru import logger

In [ ]:
# export
class LitModel(pl.LightningModule):
    def __init__(
        self, channels, width, height, num_classes, hidden_size=64, learning_rate=2e-4
    ):

        super().__init__()

        self.save_hyperparameters()

        # We take in input dimensions as parameters and use those to dynamically build model.
        self.channels = channels
        self.width = width
        self.height = height
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.mean(torch.tensor([x["loss"] for x in outputs]))

        self.log("train_loss", loss)

    def validation_step(self, batch, batch_idx):

        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        return {"val_loss": loss, "val_acc": acc}

    def validation_epoch_end(self, outputs):

        avg_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()

        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_acc", avg_acc, prog_bar=True)

        logger.info(f"val_loss: {avg_loss}, val_acc: {avg_acc}")

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)

        acc = accuracy(preds, y)

        return {"test_acc": acc}

    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()

        self.log("test_acc", avg_acc)

        logger.info(f"test_acc: {avg_acc}")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            A.Resize(32, 32),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(*dm.size(), dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params


2020-12-14 17:22:37.069 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.293164014816284, val_acc: 0.125


2020-12-14 17:22:47.258 | INFO     | __main__:validation_epoch_end:63 - val_loss: 1.6805633306503296, val_acc: 0.41401273012161255


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-14 17:22:50.139 | INFO     | __main__:test_epoch_end:78 - test_acc: 0.41882985830307007



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4188),
 'val_acc': tensor(0.4140),
 'val_loss': tensor(1.6806)}
--------------------------------------------------------------------------------


[{'val_loss': 1.6805633306503296,
  'val_acc': 0.41401273012161255,
  'test_acc': 0.41882985830307007}]

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            A.PadIfNeeded(min_height=36, min_width=36),
            A.RandomCrop(height=32, width=32),
            A.HorizontalFlip(),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            A.Cutout(num_holes=4),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(*dm.size(), dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params


2020-12-14 17:23:03.160 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.316305637359619, val_acc: 0.078125


2020-12-14 17:23:20.031 | INFO     | __main__:validation_epoch_end:63 - val_loss: 1.7644908428192139, val_acc: 0.37778663635253906


1

In [ ]:
trainer.test(model, datamodule=dm)

[autoreload of fractal_augmentation.data.cifar failed: Traceback (most recent call last):
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/s

Files already downloaded and verified


/home/shadowleaf/anaconda3/envs/dev-ink/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-12-14 17:23:40.253 | INFO     | __main__:test_epoch_end:78 - test_acc: 0.3987619876861572



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.3988),
 'val_acc': tensor(0.3778),
 'val_loss': tensor(1.7645)}
--------------------------------------------------------------------------------


[{'val_loss': 1.7644908428192139,
  'val_acc': 0.37778663635253906,
  'test_acc': 0.3987619876861572}]

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            MandelBrotFractalTransform(n_iter=2, deg=2, delta=0.1),
            A.Resize(64, 64),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(3, 64, 64, dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 394 K 
-------------------------------------
394 K     Trainable params
0         Non-trainable params
394 K     Total params
INFO:lightning:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 394 K 
-------------------------------------
394 K     Trainable params
0         Non-trainable params
394 K     Total params


2020-12-14 19:13:49.541 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.3048267364501953, val_acc: 0.15625


2020-12-14 19:18:44.080 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.0835177898406982, val_acc: 0.21218152344226837


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


2020-12-14 19:12:46.892 | INFO     | __main__:test_epoch_end:78 - test_acc: 0.18130990862846375



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.1813),
 'val_acc': tensor(0.3055),
 'val_loss': tensor(1.9136)}
--------------------------------------------------------------------------------


[{'val_loss': 1.9136290550231934,
  'val_acc': 0.30553343892097473,
  'test_acc': 0.18130990862846375}]

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 01_model.dummy.ipynb.
Converted 01a_model.resnet18.ipynb.
Converted 0a_data.cifar.ipynb.
Converted dev_setup.ipynb.
Converted index.ipynb.
